## Loading sakila data

In [15]:
import duckdb
from pathlib import Path

duckdb_path = "data/sakila.duckdb"
Path(duckdb_path).unlink(missing_ok=True)

with duckdb.connect(duckdb_path) as conn, open("sql/load_sakila.sql") as ingest_script:
    conn.sql(ingest_script.read())


In [ ]:
conn = duckdb.connect(duckdb_path)

## Which movies is longer than 180 minutes

In [36]:

films_df = conn.sql("""
SELECT
    title,
    length
FROM
    film
WHERE length >= 180; 
""").df()

films_df.head(5)

,title,length
0,ALLEY EVOLUTION,180
1,ANALYZE HOOSIERS,181
2,BAKED CLEOPATRA,182
3,CATCH AMISTAD,183
4,CHICAGO NORTH,185


In [39]:
len(films_df)

46

## There is 46 movies thar are loonger or equal to 180 minutes

## Movies that contains "love"